In [18]:
import sys
sys.path.append(r"D:\ig_pipeline")

In [19]:
from b1k_pipeline.utils import PipelineFS, get_targets

In [20]:
pipeline_fs = PipelineFS()

In [ ]:
import json
from tqdm.notebook import tqdm
from fs.zipfs import ZipFS

problematic_targets = []
for tgt in tqdm(get_targets("combined")):
    target_fs = pipeline_fs.target_output(tgt)
    assert target_fs.exists("object_list.json")
    mesh_list = json.loads(target_fs.readtext("object_list.json"))["meshes"]
    
    if not target_fs.exists("meshes.zip"):
        problematic_targets.append(tgt)
        print(tgt, "missing meshes file")
        continue
        
    missing_meshes = []
    with target_fs.open("meshes.zip", "rb") as zip_file, ZipFS(zip_file) as zip_fs:
        for mesh in mesh_list:
            if not zip_fs.exists(mesh):
                missing_meshes.append(mesh)

    if missing_meshes:
        problematic_targets.append(tgt)

  0%|          | 0/1987 [00:00<?, ?it/s]

objects/valentine_wreath-ik missing meshes file
objects/wood-we missing meshes file
scenes/Beechwood_0_garden missing meshes file
scenes/Merom_0_garden missing meshes file
scenes/Pomaria_0_garden missing meshes file
scenes/Rs_garden missing meshes file
scenes/Wainscott_0_garden missing meshes file
scenes/school_biology missing meshes file
scenes/school_chemistry missing meshes file
scenes/school_computer_lab_and_infirmary missing meshes file
scenes/school_geography missing meshes file


In [22]:
len(problematic_targets)

13

In [23]:
problematic_targets

['objects/valentine_wreath-ik',
 'objects/wood-we',
 'scenes/Beechwood_0_garden',
 'scenes/Merom_0_garden',
 'scenes/Pomaria_0_garden',
 'scenes/Rs_garden',
 'scenes/Wainscott_0_garden',
 'scenes/house_double_floor_lower',
 'scenes/office_cubicles_right',
 'scenes/school_biology',
 'scenes/school_chemistry',
 'scenes/school_computer_lab_and_infirmary',
 'scenes/school_geography']

In [24]:
print("dvc repro -s -f", " ".join(f"export_meshes@{tgt}" for tgt in problematic_targets))

dvc repro -s -f export_meshes@objects/valentine_wreath-ik export_meshes@objects/wood-we export_meshes@scenes/Beechwood_0_garden export_meshes@scenes/Merom_0_garden export_meshes@scenes/Pomaria_0_garden export_meshes@scenes/Rs_garden export_meshes@scenes/Wainscott_0_garden export_meshes@scenes/house_double_floor_lower export_meshes@scenes/office_cubicles_right export_meshes@scenes/school_biology export_meshes@scenes/school_chemistry export_meshes@scenes/school_computer_lab_and_infirmary export_meshes@scenes/school_geography


In [8]:
import yaml, collections
with open(r"D:\ig_pipeline\dvc.lock", "r") as f:
    lock = yaml.load(f, Loader=yaml.SafeLoader)

ValueError: too many values to unpack (expected 2)

In [9]:
ctr = collections.Counter()
for stage_name, stage in lock["stages"].items():
    if not stage_name.startswith("export_meshes@"):
        continue
        
    pyfile, = [x for x in stage["deps"] if x["path"].endswith("export_meshes.py")]
    md5 = pyfile["md5"]
    size = pyfile["size"]
    ctr[(md5, size)] += 1
print(ctr.most_common())

[(('69f298971e31f767a0e0645d8c2543cc', 21401), 1934), (('f629a4d2e54b32d291fd55550630cb79', 21402), 51)]


In [17]:
import hashlib
with open(r"D:\ig_pipeline\b1k_pipeline\max\export_meshes.py", "r") as f:
    print(hashlib.md5(f.read().encode("utf-8")).hexdigest())

f629a4d2e54b32d291fd55550630cb79
